<a href="https://colab.research.google.com/github/Ryansyaaw/Machine-Learning_2023/blob/main/week%2010/week10.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import library

In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

tfds.disable_progress_bar()



In [2]:
def plot_graphs(history, metric):
	plt.plot(history.history[metric])
	plt.plot(history.history['val_'+metric], '')
	plt.xlabel("Epochs")
	plt.ylabel(metric)
	plt.legend([metric, 'val_'+metric])

download dataset

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [4]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


acak data untuk pelatihan dan membuat kumpulan pasangan

In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b'A vampire\'s\'s henchman wants to call her after falling in love with a five-dollar hooker in this extremely low-budget horror-comedy. I can\'t explain all the positive comments on this movie. I\'ll chalk it up to mass hallucination, but it\'s disconcerting none the less. The one redeeming factor (and this is me being extremely generous here) might be the Grandfather who\'s the only semi-likable character in this whole mess. Don\'t waste your money, or time. In fact here\'s a word of advice, If Troma puts it out on DVD, but does NOT make it themselves, in all likelihood it\'s crap.<br /><br />Troma DVD Extras:Commentary with Omar and Kirk; second commentary with cast and crew deleted scenes; bloopers; troma interactivity; radiation march; Clip from "Terror Firmer"; Theatrical trailer ;Trailers for "the Rowdy Girls", "Teenage Catgirls in Heat", "Cannible: The Musical", and "Toxic Avenger 4" <br /><br />My Grade: D'
 b"I read Rice's novel with interest, and became quite enchan

**Buat Teks Encoder**

Membuat lapisan TextVectorization

In [6]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

Metode .adapt mengatur kosakata lapisan. 20 token pertama dapat dilihat dengan kode berikut. Setelah padding dan token yang tidak diketahui, mereka diurutkan berdasarkan frekuensi:

In [7]:
vocab=np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

Setelah kosakata diatur, lapisan dapat mengkodekan teks ke dalam indeks. Tensor indeks diberi bantalan 0 ke urutan terpanjang dalam batch (kecuali jika Anda menetapkan output_sequence_length tetap):

In [8]:
encoded_example=encoder(example)[:3].numpy()
encoded_example

array([[  4,   1,   1, ...,   0,   0,   0],
       [ 10, 321,   1, ...,   0,   0,   0],
       [ 16,   2,  86, ...,   0,   0,   0]])

In [9]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'A vampire\'s\'s henchman wants to call her after falling in love with a five-dollar hooker in this extremely low-budget horror-comedy. I can\'t explain all the positive comments on this movie. I\'ll chalk it up to mass hallucination, but it\'s disconcerting none the less. The one redeeming factor (and this is me being extremely generous here) might be the Grandfather who\'s the only semi-likable character in this whole mess. Don\'t waste your money, or time. In fact here\'s a word of advice, If Troma puts it out on DVD, but does NOT make it themselves, in all likelihood it\'s crap.<br /><br />Troma DVD Extras:Commentary with Omar and Kirk; second commentary with cast and crew deleted scenes; bloopers; troma interactivity; radiation march; Clip from "Terror Firmer"; Theatrical trailer ;Trailers for "the Rowdy Girls", "Teenage Catgirls in Heat", "Cannible: The Musical", and "Toxic Avenger 4" <br /><br />My Grade: D'
Round-trip:  a [UNK] [UNK] wants to call her after [UNK] in

**Buat Model**

In [10]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [11]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


Untuk memastikan bahwa ini berfungsi seperti yang diharapkan, evaluasi sebuah kalimat dua kali. Pertama, satu kalimat sehingga tidak ada bantalan (padding) untuk disamarkan:

In [12]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

1/1 [==============================] - 3s 3s/step
[0.00480865]


Sekarang, evaluasi lagi dalam batch dengan kalimat yang lebih panjang. Hasilnya harus sama:

In [13]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

1/1 [==============================] - 0s 322ms/step
[0.00480864]


compile model Keras untuk mengonfigurasi proses pelatihan:

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

**Train Model**

In [15]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

Epoch 1/10
 17/391 [>.............................] - ETA: 10:01 - loss: 0.6933 - accuracy: 0.5119

KeyboardInterrupt: ignored